## 03_간단한 사이트 데이터 수집

- View Page Source(소스보기)를 통해 서버로 부터 받아오는 소스에 데이터가 있는 것이 확인이 된 경우 
- requests 모듈을 통해 요청한다.
- bs4를 통해 분석해서 수집한다.
- https://pjt3591oo.github.io/

In [1]:
# Package 리스트 보기
# !pip list

### 필요모듈 설치

In [2]:
!pip install numpy

In [3]:
!pip install pandas

In [4]:
!pip install bs4

In [5]:
!pip install requests

In [6]:
!pip install lxml

### import

In [7]:
# 수집한 데이터 저장용
import numpy as np
import pandas as pd
# 요청
import requests
# 수집
import bs4
# 기타
import time
import os
from IPython.display import clear_output

In [8]:
# 요청함수
# 요청 후 서버가 보낸 결과를 반환한다.
def getSource(site) :
    # 헤더 정보 셋팅
    # user_agent : 웹브라우저가 서버로 보내는 문자열이고 
    # 서버는 이를 통해 브라우저 정보나 컴퓨터 정보를 파악한다.
    # 일부 사이트는 user_agent가 전달되지 않으면 데이터를 전달하지 않는 경우도 있다. 
    # 이에 user-agent를 셋팅해서 지금 요청한 도구가 python 코드가 아닌 일반 웹브라우저인것처럼 속일 수 있다.
    # http://m.avalon.co.kr/check.html 사이트에서 user-agent 값을 확인하여 넣어준다.
    
    header_info = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'
    }
    
    # 요청한다
    response = requests.get(site, headers=header_info)
    return response.text.strip()



In [17]:
# 한 페이지의 데이터를 수집해 저장하는 함수
def getData(source, filename) :
    
    # 데이터 있는 전체를 가져온다.
    a1 = source.select_one('body > main > div > div') # select_one: 하나가지고 오는 함수
    # print(a1)
    
    # 전체에서 데이터 항목들을 가져온다.
    # 제일 마지막은 제외한다.
    a2 = a1.select('div')[:-1] # select: 여러개 가지고 오는 함수, [:-1] 마지막 빼고 출력
    # print(a2)
    
    data_dict = {
        '큰 제목' : [],
        '작은 제목' : [],
        '날짜' : [],
        '작성자' : []
    }
    
    # 항목의 수 만큼 반복한다.
    for div in a2 :
        # print(div)

        # 큰 제목
        a3 = div.select_one('h3 > a')
        data1 = a3.text.strip()
        # print(data1)
        
        # 작은 제목
        a4 = div.select_one('h4')
        data2 = a4.text.strip()
        #print(data2)
        
        # 날짜와 작성자
        a5 = div.select_one('p > span')
        str5 = a5.text.strip()
        str6 = str5.split('|')
        data3 = str6[0].strip()
        data4 = str6[1].strip()
        # print(data3)
        # print(data4)
        
#         print('----------------------')

        # 데이터를 담는다.
    
        # 길이가 0인 문자열은 결측치로 변경한다.
        if len(data1) == 0 :
            data1 = np.nan
            
        if len(data2) == 0 :
            data2 = np.nan
            
        if len(data3) == 0 :
            data3 = np.nan
            
        if len(data4) == 0 :
            data4 = np.nan
            
        data_dict['큰 제목'].append(data1)
        data_dict['작은 제목'].append(data2)
        data_dict['날짜'].append(data3)
        data_dict['작성자'].append(data4)
        
    # 데이터 프레임을 생성한다.
    df1 = pd.DataFrame(data_dict)
    display(df1)
    
    # 파일이 없다면 : 첫 번째 저장
    if os.path.exists(fileName) == False :
        df1.to_csv('data1.csv', encoding='utf-8-sig', index=False)
    # 파일이 있다면..
    else :
        df1.to_csv('data1.csv', encoding='utf-8-sig', index=False, mode='a', header=None) # mode='a': APPEND

### 페이지 1에 있는 글 목록들 df로 보여줌
```python
a1 = getSource('https://pjt3591oo.github.io/')
# bs4 객체로 생성한다.
soup = bs4.BeautifulSoup(a1, 'lxml')
# 데이터를 수집해 저장한다.
getData(soup)
```

In [14]:
# 다음 페이지 주소를 가져오는 함수
def getNextPage(source) :
    
    # Next 버튼 태그를 가져온다.
    a1 = source.select_one('body > main > div > div > div.pagination > ul > li:nth-child(6) > a')
    # print(a1)
    
    # 태그가 존재 한다면...
    if a1 != None :
        # href 속성값을 가져온다.
        href = a1.attrs['href']
        return href
    # 태그가 존재하지 않는다면..
    else :
        return None

In [15]:
# 페이지의 주소
site = 'https://pjt3591oo.github.io'
nextPage = ''
fileName = 'data1.csv'

# 파일이 존재하면 삭제한다.
if os.path.exists(fileName) == True :
    os.remove(fileName)

while True :
    # 딜레이
    time.sleep(1)
    
    # 화면 청소
    clear_output(wait=True)
    
    # 페이지 요청
    print(f'{site}{nextPage} 수집 중...')
    a1 = getSource(f'{site}{nextPage}')
    source = bs4.BeautifulSoup(a1)

    # 현재 페이지의 데이터를 수집한다.
    getData(source, fileName)
    
    # 다음 페이지의 주소를 가져온다.
    nextPage = getNextPage(source)
    
    # 다음 페이지가 None이면 반복을 중단한다.
    if nextPage == None :
        print('수집완료')
        break

https://pjt3591oo.github.io/page4/ 수집 중...


,큰 제목,작은 제목,날짜,작성자
0,[intro] Welcome to mung,NaN,2017-04-04 13:10:05 +0000,박정태


수집완료


In [16]:
# 복원하는 코드
pd.read_csv('data1.csv')

,큰 제목,작은 제목,날짜,작성자
0,[programming] [react] react 작업환경 설,react를 시작하기 전에 환경셋팅을 해보자,2017-05-20 06:29:05 +0000,박정태
1,[programming] drag and drop을 이용하여 파일 업로드를 해보자,"query의 외부 라이브러리가 아닌 drag, drop 이벤트를 활용하여 기능 구현해보기",2017-05-09 23:47:05 +0000,박정태
2,"[database] mysqldump를 이용하여 데이터 백업, 복원하기",mysqldump를 이용하여 디비 백업과 source를 이용하여 데이터 복원을 해보자,2017-05-04 05:33:05 +0000,박정태
3,[database] mysql 원격접속하는 방법,"mysql 디비설정, 유저설정을 통해 원격접속",2017-05-03 11:43:05 +0000,박정태
4,"[node.js] 파일 리더기 만들기 - 사용 모듈 정리, pdf와 hwp 구조","docx, hwp, pdf 파일",2017-05-01 06:02:05 +0000,박정태
5,"[node.js] 파일 리더기 만들기 - pdf를 html로 변환, docx를 pd...",pdf파일 html로 변경하기,2017-04-30 15:19:05 +0000,박정태
6,[programming] git 원격 저장소 바꾸기,remote set-url을 이용하여 원격 저장소를 바꾸자,2017-04-25 13:09:05 +0000,박정태
7,[programming] python working directory를 바꿔보자,working directory설정을 하여 경로 문제를 해결하자,2017-04-25 09:19:05 +0000,박정태
8,[server] docker commends,도커의 명령어들을 간단하게 알아보기,2017-04-24 23:19:05 +0000,박정태
9,[programming] python utc를 timestamp로 바꾸는 방법,python에서 utc를 timestamp로 바꾸는 방법,2017-04-24 13:19:05 +0000,박정태
